In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy
import requests
import re
import json
import urllib.request

## Scrape event information for a driver

In [2]:
def events_for_a_driver(my_url):
    
    # Load races on url
    raw_html = requests.get(my_url).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    events = soup_doc.find_all("div", class_="profile-start-line d-flex justify-content-start w-100 lh-120 py-1 flex-wrap")
    races = []
    
    # Find all WRC races
    events = [e for e in events if e.span.text == ' WRC']

    # List out all informations
    
    for e in events:
        race = {}
        race['Driver'] = soup_doc.find("h4").a.text.strip()
        race['Name'] = e.find("a", title="Show entry info and stats").text
        URL = e.find("a", title="Show entry info and stats")['href']
        race['URL'] = URL
    # Check the URL
        print(URL)
        id1 = re.findall("/entryinfo/(\d+)",URL)[0]
        id2 = re.findall("/entryinfo/.+/(\d+)",URL)[0]
        Year = re.findall("/entryinfo/.+-(\d\d\d\d)/\d+/",URL)[0]
        race['Year'] = Year
        
        
        try:
            rank = e.find("span", class_="fs-09 font-weight-normal text-secondary").text
            ra = re.findall("(\d).",rank)[0]
            race['Race_Result'] = ra
            race['DNG_Reason'] = "Null"
        except:
            ra = e.find("div", class_="profile-start-ret text-danger font-weight-bold")
            race['Race_Result'] = "DNF"
            race['DNG_Reason'] = ra
        
        race['Php_url'] = "https://www.ewrc-results.com/inc/json_driverinfo.php?event="+id1+"&entry="+id2+"&sct=&cat=&lg=en"
        
        print("Php found" + race['Php_url'])
        
        x = urllib.request.urlopen(race['Php_url'])
        raw_data = x.read()
        encoding = x.info().get_content_charset('utf8')  # JSON default
        ssresult = json.loads(raw_data.decode(encoding))
        print(ssresult, len(ssresult['stages'][0][1]))

        
        for idx, pos in enumerate(ssresult['pos'][0][1]):
            
            stage = race.copy()
            
            stage['p_overall'] = pos      
            stage['stage'] = ssresult['stages'][0][1][idx]
            stage['ssname'] = ssresult['sslist'][idx]
            races.append(stage)
            # print(ssresult['pos'][0][1])
        
        
        print("Php info scraped.")
        
        
    return races



In [3]:
races = events_for_a_driver('https://www.ewrc-results.com/profile/456-colin-mcrae/')

/entryinfo/13-rally-of-turkey-2006/811/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=13&entry=811&sct=&cat=&lg=en
{'pos': [['Overall', [5, 4, 4, 5, 7, 6, 6, 7, 7, 8, 8, 7, 7, 7, 7]]], 'stages': [['Stage results', [5, 4, 3, 9, 9, 2, 7, 13, 8, 11, 5, 9, 2, 10, 9]]], 'sslist': ['SS3', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/48-telstra-rally-australia-2005/1416/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=48&entry=1416&sct=&cat=&lg=en
{'pos': [['Overall', [8, 7, 10, 6, 6, 5, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3]]], 'stages': [['Stage results', [8, 7, 13, 2, 12, 5, 4, 7, 4, 9, 6, 5, 3, 5, 3, 3, 3, 3, 7, 8, 5, 4, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22',

{'pos': [['Overall', [3, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [3, 2, 3, 10, 6, 1, 2, 4, 2, 1, 8, 4, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14'], 'colors': ['#e2431e', '#f1ca3a']} 13
Php info scraped.
/entryinfo/93-network-q-rally-of-great-britain-2002/4774/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=93&entry=4774&sct=&cat=&lg=en
{'pos': [['Overall', [3, 17, 10, 8, 7, 7, 8, 8, 6, 6, 6, 7, 6, 6, 5, 5]]], 'stages': [['Stage results', [3, 21, 5, 3, 5, 5, 7, 7, 4, 8, 5, 49, 6, 5, 6, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/92-telstra-rally-australia-2002/4991/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=92&entry=4991&sct=&cat=&lg=en
{'pos': [['Overall', [9, 7, 9, 8, 8, 7, 6, 6, 6, 6, 

{'pos': [['Overall', [9, 17, 15, 12, 9, 8, 7, 7, 7, 9, 12, 12, 12, 11, 11]]], 'stages': [['Stage results', [9, 21, 10, 8, 9, 3, 1, 7, 14, 24, 42, 8, 9, 8, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/104-rallye-sanremo-rallye-ditalia-2001/6548/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=104&entry=6548&sct=&cat=&lg=en
{'pos': [['Overall', [7, 13, 13, 13, 13, 13, 13, 13, 12, 11, 10, 10, 10, 10, 10, 10, 10, 10, 9, 8]]], 'stages': [['Stage results', [7, 13, 11, 13, 12, 12, 16, 14, 9, 7, 27, 8, 9, 6, 9, 8, 10, 16, 7, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20'], 'colors': ['#e2431e', '#f1ca3a']} 20
Php info scraped.
/entryinfo/103-propecia-rally-new-zealand-2001/6481/
Php foundhttps://www.ewrc-res

{'pos': [['Overall', [12, 9, 8, 7, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2]]], 'stages': [['Stage results', [12, 6, 8, 5, 6, 2, 5, 6, 7, 7, 4, 6, 6, 1, 5, 5, 3, 3, 1, 1, 3, 6, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/115-propecia-rally-new-zealand-2000/8010/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=115&entry=8010&sct=&cat=&lg=en
{'pos': [['Overall', [3, 5, 5, 6, 7, 7, 7, 6, 6, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [3, 9, 8, 12, 11, 7, 1, 1, 2, 2, 2, 2, 1, 1, 3, 3, 2, 4, 7, 1, 1, 1, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24'], 'colors':

{'pos': [['Overall', [1, 7, 6, 6, 37, 39, 36, 33, 29, 22, 21, 19, 21, 16]]], 'stages': [['Stage results', [1, 7, 4, 2, 74, 46, 7, 12, 6, 1, 2, 4, 44, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 14
Php info scraped.
/entryinfo/125-tap-rallye-de-portugal-1999/10414/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=125&entry=10414&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 1, 3, 1, 1, 3, 7, 1, 3, 7, 3, 4, 5, 7, 4, 4, 4, 2, 5, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 21
Php info scraped.
/entryinfo/124-555-safari-rally-1999/8242/
Php foundhttps://www.ewrc-resul

{'pos': [['Overall', [3, 7, 5, 6, 5, 5, 5, 6, 5, 5, 5, 5, 4, 4, 4, 4, 3, 3]]], 'stages': [['Stage results', [3, 16, 6, 8, 2, 5, 8, 7, 2, 2, 1, 8, 7, 7, 6, 2, 3, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 18
Php info scraped.
/entryinfo/177-network-q-rac-rally-1997/12903/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=177&entry=12903&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 8, 3, 3, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 5, 6, 7, 2, 2, 1, 1, 1, 1, 14, 37, 1, 1, 1, 1, 3, 1, 4, 1, 1, 1, 3, 1, 1, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26'], 'colors': ['#e2431e', '#f1ca3a']} 26
Php info scraped.
/e

{'pos': [['Overall', []]], 'stages': [['Stage results', []]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 0
Php info scraped.
/entryinfo/477-international-swedish-rally-1996/36379/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=477&entry=36379&sct=&cat=&lg=en
{'pos': [['Overall', [9, 6, 5, 5, 4, 4, 10, 6, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3]]], 'stages': [['Stage results', [9, 4, 5, 4, 1, 10, 48, 2, 2, 2, 1, 2, 4, 12, 1, 6, 2, 1, 1, 1, 1, 6, 1, 2, 2, 2, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26', 'SS27'], 'colors': ['#e2431e', '#f1ca3a']} 27
Php info scraped.
/entryinfo/503-network-q-rac-rally-1995/37841/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=503&entry=37841&

{'pos': [['Overall', [2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [2, 3, 3, 4, 3, 7, 2, 3, 5, 6, 3, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26', 'SS27', 'SS28', 'SS29', 'SS30', 'SS31', 'SS32', 'SS33', 'SS34', 'SS35', 'SS36'], 'colors': ['#e2431e', '#f1ca3a']} 12
Php info scraped.
/entryinfo/532-tour-de-corse-rallye-de-france-1993/40828/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=532&entry=40828&sct=&cat=&lg=en
{'pos': [['Overall', [8, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]], 'stages': [['Stage results', [8, 9, 8, 5, 10, 8, 6, 7, 7, 6, 7, 6, 7, 6, 5, 6, 6, 6, 6, 6, 6, 7, 6, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', '

{'pos': [['Overall', [16, 15, 17, 16, 15, 13, 12, 12, 12, 11, 11, 10, 10, 11, 10, 10, 12, 12, 12, 12, 12, 12, 12, 11, 10, 13, 13, 12, 11, 11, 13, 13, 13, 15, 15, 15, 15]]], 'stages': [['Stage results', [16, 14, 14, 18, 13, 10, 15, 11, 15, 9, 8, 9, 10, 8, 10, 12, 16, 9, 15, 11, 12, 10, 15, 8, 15, 15, 14, 8, 8, 15, 16, 8, 14, 11, 12, 3, 10]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26', 'SS27', 'SS28', 'SS29', 'SS30', 'SS31', 'SS32', 'SS33', 'SS34', 'SS35', 'SS36', 'SS37'], 'colors': ['#e2431e', '#f1ca3a']} 37
Php info scraped.
/entryinfo/2477-lombard-rac-rally-1988/211539/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=2477&entry=211539&sct=&cat=&lg=en
{'pos': [['Overall', []]], 'stages': [['Stage results', []]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', '

In [4]:
df = pd.DataFrame(races)
df['Year'] = df['Year'].astype('int')
df_filtered = df[df['Year'] >= 2000]
df_filtered

,Driver,Name,URL,Year,Race_Result,DNG_Reason,Php_url,p_overall,stage,ssname
0,Colin McRae,Rally of Turkey,/entryinfo/13-rally-of-turkey-2006/811/,2006,DNF,[\r\n Mechanica...,https://www.ewrc-results.com/inc/json_driverin...,5,5,SS3
1,Colin McRae,Rally of Turkey,/entryinfo/13-rally-of-turkey-2006/811/,2006,DNF,[\r\n Mechanica...,https://www.ewrc-results.com/inc/json_driverin...,4,4,SS5
2,Colin McRae,Rally of Turkey,/entryinfo/13-rally-of-turkey-2006/811/,2006,DNF,[\r\n Mechanica...,https://www.ewrc-results.com/inc/json_driverin...,4,3,SS6
3,Colin McRae,Rally of Turkey,/entryinfo/13-rally-of-turkey-2006/811/,2006,DNF,[\r\n Mechanica...,https://www.ewrc-results.com/inc/json_driverin...,5,9,SS7
4,Colin McRae,Rally of Turkey,/entryinfo/13-rally-of-turkey-2006/811/,2006,DNF,[\r\n Mechanica...,https://www.ewrc-results.com/inc/json_driverin...,7,9,SS8
...,...,...,...,...,...,...,...,...,...,...
908,Colin McRae,Rallye Automobile Monte-Carlo,/entryinfo/108-rallye-automobile-monte-carlo-2...,2000,DNF,[\r\n Engine ...,https://www.ewrc-results.com/inc/json_driverin...,4,2,SS10
909,Colin McRae,Rallye Automobile Monte-Carlo,/entryinfo/108-rallye-automobile-monte-carlo-2...,2000,DNF,[\r\n Engine ...,https://www.ewrc-results.com/inc/json_driverin...,3,1,SS11
910,Colin McRae,Rallye Automobile Monte-Carlo,/entryinfo/108-rallye-automobile-monte-carlo-2...,2000,DNF,[\r\n Engine ...,https://www.ewrc-results.com/inc/json_driverin...,3,2,SS12
911,Colin McRae,Rallye Automobile Monte-Carlo,/entryinfo/108-rallye-automobile-monte-carlo-2...,2000,DNF,[\r\n Engine ...,https://www.ewrc-results.com/inc/json_driverin...,3,3,SS13


In [5]:
df_filtered.to_csv('Colin_McRae.csv')